In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.ndimage import gaussian_filter1d
import json
from scipy.special import expit
from scipy import signal

In [2]:
data = pd.read_csv('../raw_data/STIM_5_6_SIZEDOT_0.004_0.025/zfdata0.csv')
data

,X,Y,ANGLE,CUM_angle,TAIL_P1,TAIL_P2,TAIL_P3,TAIL_P4,TAIL_P5,TAIL_P6,...,L_EYE,R_EYE,Timestamp,STIM_type,SIZE_dot,DIST_dot,AMPL_rot,SPEED_rot,LUM_dot,BGLUM
0,367.0252,593.7792,1.177578,1.177578,0,0.137511,0.124135,0.016308,0.016309,0.056841,...,-0.423145,0.391471,261,0,0.000,0.1,1.570797,0.35,1,0
1,366.7211,594.5103,1.187335,1.187335,0,0.141481,0.141480,0.020278,0.020277,0.074310,...,-0.416073,0.392175,278,0,0.000,0.1,1.570797,0.35,1,0
2,367.0742,593.7476,1.176733,1.176733,0,0.141743,0.141743,0.020540,0.020540,0.061074,...,-0.413420,0.389333,293,0,0.000,0.1,1.570797,0.35,1,0
3,367.1762,593.4625,1.177119,1.177119,0,0.113289,0.046117,0.046117,0.019111,0.019112,...,-0.438036,0.370654,309,0,0.000,0.1,1.570797,0.35,1,0
4,367.0626,593.7814,1.176675,1.176675,0,0.141862,0.128486,0.020660,0.047685,0.061192,...,-0.415686,0.386214,325,0,0.000,0.1,1.570797,0.35,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24843,499.3449,665.0624,0.438917,-0.438917,0,-0.053205,0.068262,0.027816,-0.012716,0.027817,...,-0.313755,0.351815,317,6,0.004,0.1,1.570797,0.35,1,0
24844,499.3401,665.0195,0.437999,-0.437999,0,-0.039478,0.055041,0.014552,-0.012473,0.014552,...,-0.306537,0.363862,335,6,0.004,0.1,1.570797,0.35,1,0
24845,499.3416,665.0223,0.438048,-0.438048,0,-0.035969,0.072014,0.018062,-0.008963,0.031570,...,-0.299882,0.364554,353,6,0.004,0.1,1.570797,0.35,1,0
24846,499.3479,665.0674,0.438999,-0.438999,0,-0.045899,0.048620,-0.018893,0.008131,0.008132,...,-0.315580,0.355105,373,6,0.004,0.1,1.570797,0.35,1,0


In [ ]:
def find_vergence_event_threshold(dataframe):
    
    
    
    
    threshold = 
    return threshold